In [3]:
import glob
import os
import pandas as pd
from matplotlib import pyplot as pyplot
import json


In [ ]:
txt_files = glob.glob("problem-*.txt", root_dir = "data/pan")
json_files = glob.glob("truth-problem-*.json", root_dir="data/pan")

for (txt_path, json_path) in zip(txt_files,json_files):
    with open(json_path) as json_f:
        json_dict = json.load(json_f)
    with open(txt_path) as txt_f:
        txt_content = txt_f.read()
